In [ ]:
import gurobipy as gp
import numpy as np
import matplotlib.pyplot as plt
import random
from random import randint
from sklearn.datasets import make_blobs
from utility import *
import os

# P-Mediana

Soluzione del problema di clustering ottenuta risolvendo all'esatto, tramite il solutore gurobi, il problema di programmazione lineare intera di p-mediana.

Il programma, dopo aver caricato l'istanza di un problema di clustering, calcolate tutte le distanze $d_{i,j}$ tra tutte le coppie di punti, definisce il modello di PLI e procede a risolverlo.

Calcolo delle distanze, necessarie per la definizione del modello di PLI.

In [ ]:
dirname = os.path.dirname("__file__")
filename = os.path.join(dirname, 'benchmark/benchmark{8}.txt')
points = load_points(filename)

points = load_points(f'C:/Users/franc/Documents/GitHub/Ricerca_Operativa_2022/Ricerca_Operativa_2022/benchmark/benchmark{test}.txt')
n_clusters = [5,5,5,10,5,10,5,5,5,5,4,5,10,5,5]

K = n_clusters[test-1]
N = len(points)

dist = {}

for i in range(N):
    for j in range(N):
        dist[(i,j)] = np.sum((points[i]-points[j])**2)

# Formulazione del problema

### Variabili

- $y_{i}$ : Variabile binaria che descrive l'utilizzo del punto $i$ come centroide di un cluster.

- $x_{i,j}$: Variabile binaria che determina se il punto $j$ è raggruppato nel cluster con centroide $i$. Queste variabili possono essere in realtà modellate come intera data la totale unimodularità della matrice dei vincoli.
### Funzione obiettivo
Minimizza la distanza di ogni punto dal proprio centroide:

$$
\text{Minimize} \quad  \sum_{i=1}^N\sum_{j=1}^N d_{i,j}x_{i,j} 
$$

### Vincoli
- Esattamente K punti devono essere scelti come centroidi:

$$
  \sum_{i = 1}^N y_{i} = K \quad \quad
$$

- Ogni punto $j$ deve essere connesso ad un centroide:

$$
  \sum_{i = 1}^N x_{i,j} = 1 \quad \quad i \in \{1,2,...,N\}
$$

- Un punto $j$ può essere connesso ad un altro $i$ solo se $i$ è stato scelto come centroide:

$$
  0 \le x_{i,j} \le y_i \quad \quad i,j \in \{1,2,...,N\}
$$

In [ ]:
model = gp.Model()
x = model.addVars(N,N, name = "X") #i elemento , j centroide
y = model.addVars(N, vtype = gp.GRB.BINARY, name = "Y")

obj = model.setObjective(x.prod(dist), gp.GRB.MINIMIZE)

model.addConstrs((sum(x[i,j] for j in range(N)) == 1 for i in range(N)))
model.addConstr(sum(y[j] for j in range(N))==K)
model.addConstrs((x[i,j]<=y[j] for i in range(N) for j in range(N)))

model.optimize()


Valutazione della soluzione ottenuta tramite la metrica di squared inner distance e stampa della soluzione

In [ ]:
centroids = []
for j in range(N):
    if y[j].X == 1:
        centroids.append(j)

sol = np.zeros(N)

for i in range(N):
    for cluster,j in enumerate(centroids):
        if x[i,j].X == 1:
            sol[i] = cluster


print("{}".format(squared_inner_distance(sol, points, K)))

printR2sol(points, sol, K)